In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0].split('Documents')[0],'Documents/BecaNLP/Utils'))

import NLPUtils as nlp
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package punkt to /home/lestien/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Cargamos el dataset

In [40]:
df = pd.read_csv('../train.csv',sep = '|')
df['Intencion'] = df.Intencion.str.findall(r'\d+').apply(lambda x: int(x[0]))

# # Me quedo con las n categorías más frecuentes:
# n = 20
# counts_values = df['Intencion'].value_counts()
# keep_cats = counts_values.loc[counts_values >= n].index.tolist()
# df = df.loc[df['Intencion'].isin(keep_cats)]
# df.head()

# Clasificación por Naive Bayes

## Resultados importantes:

* Los resultados no mejoran si saco clases.
* No cambia si `fit_prior=False`
* Con Bernoulli classifier da siempre lo mismo. No sé por qué.

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from utils import *

ngram_range = (1,2)
max_features_list = np.linspace(5000,80000,10)
max_features_list = [None]
#alphas = np.logspace(.00001,.01,10) - 1
alpha = 0.002
thresholds = [1., 5., 10., 20., 50., 100.]

for threshold in thresholds:
    vec = CountVectorizer(ngram_range=ngram_range,max_features=None,token_pattern=r'\b\w+\b')
    #model = MultinomialNB(alpha=alpha,fit_prior=False)
    model = BernoulliNB(alpha=alpha,binarize=threshold)

    scores = k_fold_validation(model,df,vec,reweight=None,k_folds=10,random_state=0,metrics=['accuracy','balanced_accuracy'])
    #print('alpha = {:.5g}'.format(alpha))
    print('threshold = {}'.format(threshold))
    print(scores)
    

/home/lestien/anaconda3/envs/nlpenv-gpu/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/lestien/anaconda3/envs/nlpenv-gpu/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/lestien/anaconda3/envs/nlpenv-gpu/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/lestien/anaconda3/envs/nlpenv-gpu/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/lestien/anaconda3/envs/nlpenv-gpu/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred con

threshold = 1.0
{'accuracy': 0.046458038004903376, 'balanced_accuracy': 0.008075005783888964}
threshold = 5.0
{'accuracy': 0.02984446242185393, 'balanced_accuracy': 0.003269783057419788}
threshold = 10.0
{'accuracy': 0.02984446242185393, 'balanced_accuracy': 0.003269783057419788}
threshold = 20.0
{'accuracy': 0.02984446242185393, 'balanced_accuracy': 0.003269783057419788}
threshold = 50.0
{'accuracy': 0.02984446242185393, 'balanced_accuracy': 0.003269783057419788}
threshold = 100.0
{'accuracy': 0.02984446242185393, 'balanced_accuracy': 0.003269783057419788}


# Evaluamos en Test

In [44]:
df_train = df
df_test = pd.read_csv('../test_santander.csv')

ngram_range = (1,2)
max_features = None
alpha = 0.0002
vec = CountVectorizer(ngram_range=ngram_range, max_features=max_features,token_pattern=r'\b\w+\b')

model = MultinomialNB(alpha=alpha)

X_train = vec.fit_transform(df_train['Pregunta'])
X_test = vec.transform(df_test['Pregunta'])

model.fit(X_train,df_train['Intencion'].values)
y_pred = model.predict(X_test)
df_test['Pregunta'] = y_pred
df_test.to_csv('./results.csv',index=False,header=False)